In [104]:
from qiskit.circuit import QuantumCircuit, ParameterVector, QuantumRegister
from qiskit.circuit.library import RXGate, UGate, RZXGate
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix, random_statevector, Choi, Operator, SuperOp, SparsePauliOp, Pauli
import numpy as np

from rl_qoc import QuantumEnvironment, QiskitConfig, QEnvConfig, ExecutionConfig, StateTarget, ShadowReward, GateTarget

In [ ]:
def shadow_bound_state(error, observables, failure_rate=0.01):
   
    M = len(observables)
    K = 2 * np.log(2 * M / failure_rate)
    shadow_norm = (
        lambda op: np.linalg.norm(
            op - np.trace(op) / 2 ** int(np.log2(op.shape[0])), ord=np.inf
        )
        ** 2
    )
    N = 34 * max(shadow_norm(o) for o in observables) / error ** 2
    return max(int(np.ceil(N * K)), 10000), int(K), M           #sometimes N = 0. A limit of 10000 is set to prevent this


In [3]:
def shadow_bound_state2(error, observables, coeffs, failure_rate=0.01):
   
    M = len(observables)
    K = 2 * np.log(2 * M / failure_rate)
    shadow_norm = (
        lambda op: np.linalg.norm(
            op - np.trace(op) / 2 ** int(np.log2(op.shape[0])), ord=np.inf
        )
        ** 2
    )
    N = 34 * max(shadow_norm(observables[i]) * coeffs[i]**2 for i in range(len(observables))) / error ** 2
    
    return max(int(np.ceil(N.real * K)), 10000), int(K), M           #sometimes N = 0. A limit of 10000 is set to prevent this


In [ ]:
# TEST 1: 2 qubits, 1 parameter only

# simplified 2 qubit circuit of one parameter
def apply_parametrized_gate(qc: QuantumCircuit, params: ParameterVector, qr: QuantumRegister, *args, **kwargs):
    #test circuit 1
    qc.ry(params[0], 0)
    qc.cx(0,1)

# 2 qubit parametrized bell state of one parameter
theta = np.pi #generate a random target state; this is the goal we want to obtain
tgt_state = (np.cos(theta/2) * Statevector.from_label('00') + np.sin(theta/2) * Statevector.from_label('11'))  
tgt_state_dm = DensityMatrix(tgt_state)
params =  np.array([[theta]])

observable_decomp = SparsePauliOp.from_operator(Operator(tgt_state_dm))
pauli_coeff = observable_decomp.coeffs   #to also be used in shadow bound
pauli_str = observable_decomp.paulis
observables = [Pauli(str).to_matrix() for str in pauli_str]
error = 0.1  # Set the error tolerance for the shadow bound state
shadow_size, partition, no_observables = shadow_bound_state2(error, observables, pauli_coeff)
print("Shadow Size, Partition, Number of Observables: ", shadow_size, partition, no_observables)

In [124]:
#TEST 2: 2 qubits, random target state
def calculate_shadow_bounds(tgt_state_dm, error):
    observable_decomp = SparsePauliOp.from_operator(Operator(tgt_state_dm))
    pauli_coeff = observable_decomp.coeffs   #to also be used in shadow bound
    pauli_str = observable_decomp.paulis
    observables = [Pauli(str).to_matrix() for str in pauli_str]
    shadow_size, partition, no_observables = shadow_bound_state2(error, observables, pauli_coeff)
    print("Shadow Size, Partition, Number of Observables: ", shadow_size, partition, no_observables)

#TEST 3: 2 qubits , random target state, no decomposition
def calculate_shadow_bounds_no_decomp(tgt_state_dm, error):
    observables = [tgt_state_dm.data]
    shadow_size, partition, no_observables = shadow_bound_state(error, observables, failure_rate=0.01)
    print("Shadow Size, Partition, Number of Observables, no decomp: ", shadow_size, partition, no_observables)

In [155]:
for n in range(1):
    no_qubits = 4
    error = 0.1
    tgt_state = random_statevector(2**no_qubits)  
    tgt_state_dm = DensityMatrix(tgt_state)
    calculate_shadow_bounds(tgt_state_dm, error)
    calculate_shadow_bounds_no_decomp(tgt_state_dm, error)

# decompose is almost always better for the bounds
print("DM dimensions: " , tgt_state_dm.data.shape)

Shadow Size, Partition, Number of Observables:  64807 21 256
Shadow Size, Partition, Number of Observables, no decomp:  109412 10 1
DM dimensions:  (16, 16)


In [130]:
# Choi State; 1 Qubit Test, ry gate
num_qubits = 1
error = 0.1

params = np.array([np.random.rand()*2* np.pi])
qc = QuantumCircuit(num_qubits)
qc.ry(2*params[0], 0)
specific_gate = qc.to_gate(label="U_entangle")
gate_target = GateTarget(specific_gate)
target_choi = Choi(gate_target.target_operator)
target_choi_dm = DensityMatrix(target_choi.data/ 2**num_qubits)
calculate_shadow_bounds(target_choi_dm, error)
calculate_shadow_bounds_no_decomp(target_choi_dm, error)
print("DM dimensions: " , target_choi_dm.data.shape)

Shadow Size, Partition, Number of Observables:  27120 14 6
Shadow Size, Partition, Number of Observables, no decomp:  44159 10 1
DM dimensions:  (4, 4)


In [131]:
# Choi State; 1 Qubit Test, randomised U gate
num_qubits = 1
error = 0.1

params = ParameterVector("theta", 3)
qc = QuantumCircuit(1)
qc.u(params[0], params[1], params[2], 0)


# Create parameterized gate
param_gate = qc.to_gate(label="U_entangle")
param_random = np.array([np.random.rand()*2* np.pi for n in range(3)])
bound_qc = qc.assign_parameters({
    params[0]: param_random[0],
    params[1]: param_random[1],
    params[2]: param_random[2],
})

specific_gate = bound_qc.to_gate(label="U_entangle")

gate_target = GateTarget(specific_gate)

target_choi = Choi(gate_target.target_operator)
target_choi_dm = DensityMatrix(target_choi.data / 2**num_qubits)
calculate_shadow_bounds(target_choi_dm, error)
calculate_shadow_bounds_no_decomp(target_choi_dm, error)
print("DM dimensions: " , target_choi_dm.data.shape)

Shadow Size, Partition, Number of Observables:  29074 15 10
Shadow Size, Partition, Number of Observables, no decomp:  50489 10 1
DM dimensions:  (4, 4)


In [157]:
# Choi State; 2 Qubit Test, randomised U gate
num_qubits = 2
error = 0.1

params = ParameterVector("theta", 6)
qc = QuantumCircuit(num_qubits)
qc.u(params[0], params[1], params[2], 0)
qc.u(params[3], params[4], params[5], 1)
qc.cx(0, 1)

# Create parameterized gate
param_gate = qc.to_gate(label="U_entangle")
param_random = np.array([np.random.rand()*2* np.pi for n in range(6)])
bound_qc = qc.assign_parameters({
    params[0]: param_random[0],
    params[1]: param_random[1],
    params[2]: param_random[2],
    params[3]: param_random[3],
    params[4]: param_random[4],
    params[5]: param_random[5],
})

specific_gate = bound_qc.to_gate(label="U_entangle")

gate_target = GateTarget(specific_gate)

target_choi = Choi(gate_target.target_operator)
target_choi_dm = DensityMatrix(target_choi.data / 2**num_qubits)
calculate_shadow_bounds(target_choi_dm, error)
calculate_shadow_bounds_no_decomp(target_choi_dm, error)
print("DM dimensions: " , target_choi_dm.data.shape)

Shadow Size, Partition, Number of Observables:  59189 19 100
Shadow Size, Partition, Number of Observables, no decomp:  110082 10 1
DM dimensions:  (16, 16)


In [145]:
#study fidelity between decomposed and original choi state - teh breakdown is correct

observable_decomp = SparsePauliOp.from_operator(Operator(target_choi_dm))
pauli_coeff = observable_decomp.coeffs   #to also be used in shadow bound
pauli_str = observable_decomp.paulis
observables = [Pauli(str).to_matrix() for str in pauli_str]
observable_total = np.zeros(observables[0].shape, dtype=complex)
for i in range(len(pauli_str)):
    observable_total += pauli_coeff[i] * observables[i]

observable_total_dm = DensityMatrix(observable_total / np.trace(observable_total))
print("Fidelity between original and decomposed DM: ", state_fidelity(target_choi_dm, observable_total_dm))

Fidelity between original and decomposed DM:  1.0


In [ ]:
# Choi State; 4 Qubit Test, randomised U gate
num_qubits = 4
error = 0.1

params = ParameterVector("theta", 3*num_qubits)
qc = QuantumCircuit(num_qubits)
qc.u(params[0], params[1], params[2], 0)
qc.u(params[3], params[4], params[5], 1)
qc.u(params[6], params[7], params[8], 2)
qc.u(params[9], params[10], params[11], 3)
qc.cx(0, 1)
qc.cx(0, 2)
qc.cx(1, 2)
qc.cx(0, 3)
qc.cx(1, 3)
qc.cx(2, 3)

# Create parameterized gate
param_gate = qc.to_gate(label="U_entangle")
param_random = np.array([np.random.rand()*2* np.pi for n in range(num_qubits*3)])
bound_qc = qc.assign_parameters({
    params[0]: param_random[0],
    params[1]: param_random[1],
    params[2]: param_random[2],
    params[3]: param_random[3],
    params[4]: param_random[4],
    params[5]: param_random[5],
    params[6]: param_random[6],
    params[7]: param_random[7],
    params[8]: param_random[8],
    params[9]: param_random[9],
    params[10]: param_random[10],
    params[11]: param_random[11],
})

specific_gate = bound_qc.to_gate(label="U_entangle")

gate_target = GateTarget(specific_gate)

target_choi = Choi(gate_target.target_operator)
target_choi_dm = DensityMatrix(target_choi.data / 2**num_qubits)
calculate_shadow_bounds(target_choi_dm, error)
calculate_shadow_bounds_no_decomp(target_choi_dm, error)
print("DM dimensions: " , target_choi_dm.data.shape)

Shadow Size, Partition, Number of Observables:  97438 28 9352
Shadow Size, Partition, Number of Observables, no decomp:  201245 10 1
DM dimensions:  (256, 256)
